In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
def set_interval_value(x, a, b):
    # function that associate to a float x, a value encoding its position with respect to the interval [a, b]
    #  the associated values are 0, 1, 2 assigned as follows:
    if x <= a: 
        return 0
    elif a < x <= b: 
        return 1
    else: 
        return 2
    
def data2color(x, y, a, b, c, d, biv_colors):
    # This function works only with a list of 9 bivariate colors,
    # because of the definition of set_interval_value()
    # x, y: lists or 1d arrays, containing values of the two variables
    #  each x[k], y[k] is mapped to an int  value xv, respectively yv, representing its category,
    # from which we get their corresponding color  in the list of bivariate colors
    if len(x) != len(y):
        raise ValueError('the list of x and y-coordinates must have the same length')
    n_colors = len(biv_colors)
    if n_colors != 9:
        raise ValueError('the list of bivariate colors must have the length eaqual to 9')
    n = 3    
    xcol = [set_interval_value(v, a, b) for v in x]
    ycol = [set_interval_value(v, c, d) for v in y]
    # index of the corresponding color in the list of bivariate colors
    idxcol = [int(xc + n*yc) for xc, yc in zip(xcol,ycol)]
    colors = np.array(biv_colors)[idxcol]
    return list(colors)

In [3]:
# Read the dataset into a pandas dataframe
dfyellow2020 = pd.read_csv("rawdata/yellow_tripdata_2020-03.csv", \
            dtype={'PULocationID': int, 'total_amount': float}, \
            usecols=['PULocationID', 'total_amount'])

dfgreen2020 = pd.read_csv("rawdata/green_tripdata_2020-03.csv", \
            dtype={'PULocationID': int, 'total_amount': float}, \
            usecols=['PULocationID', 'total_amount'])

dfyellow2019 = pd.read_csv("rawdata/yellow_tripdata_2019-03.csv", \
            dtype={'PULocationID': int, 'total_amount': float}, \
            usecols=['PULocationID', 'total_amount'])

dfgreen2019 = pd.read_csv("rawdata/green_tripdata_2019-03.csv", \
            dtype={'PULocationID': int, 'total_amount': float}, \
            usecols=['PULocationID', 'total_amount'])

In [4]:
dfyellow2020

,PULocationID,total_amount
0,88,27.80
1,193,3.80
2,246,11.75
3,151,10.56
4,79,24.35
...,...,...
3007287,137,33.77
3007288,137,41.27
3007289,137,39.90
3007290,137,22.87


In [5]:
dfzone = pd.read_csv('taxi_data/taxi_zone_lookup.csv')

In [6]:
dfzone

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NV,NaN


LocationID 264 and 265 can be dropped as they also don't have a corresponding polygon in the shapefile

In [7]:
taxi_data2020 = dfyellow2020.groupby(['PULocationID']).sum().reindex(range(1, len(dfzone)+1)).fillna(0)
green_data2020 = dfgreen2020.groupby(['PULocationID']).sum().reindex(range(1, len(dfzone)+1)).fillna(0)
taxi_data2019 = dfyellow2019.groupby(['PULocationID']).sum().reindex(range(1, len(dfzone)+1)).fillna(0)
green_data2019 = dfgreen2019.groupby(['PULocationID']).sum().reindex(range(1, len(dfzone)+1)).fillna(0)

In [8]:
def myLog(x):
    return np.log(x) if (x > 0) else 0

#taxi_data['PULocationID'] = taxi_data.index
taxi_data2020.reset_index(level=0, inplace=True)
taxi_data2020["log_total_amount"] = np.array(list(map(myLog, taxi_data2020["total_amount"])))

green_data2020.reset_index(level=0, inplace=True)
green_data2020["log_total_amount"] = np.array(list(map(myLog, green_data2020["total_amount"])))

taxi_data2019.reset_index(level=0, inplace=True)
taxi_data2019["log_total_amount"] = np.array(list(map(myLog, taxi_data2019["total_amount"])))

green_data2019.reset_index(level=0, inplace=True)
green_data2019["log_total_amount"] = np.array(list(map(myLog, green_data2019["total_amount"])))

In [9]:
taxi_data2020["green_total_amount"] = green_data2020["total_amount"]
taxi_data2020["green_log_total_amount"] = green_data2020["log_total_amount"]
taxi_data2020.rename(columns={
                    "total_amount": "yellow_total_amount",
                    "log_total_amount": "yellow_log_total_amount"
                }, inplace=True)

taxi_data2019["green_total_amount"] = green_data2019["total_amount"]
taxi_data2019["green_log_total_amount"] = green_data2019["log_total_amount"]
taxi_data2019.rename(columns={
                    "total_amount": "yellow_total_amount",
                    "log_total_amount": "yellow_log_total_amount"
                }, inplace=True)

taxi_data2020["Borough"] = dfzone["Borough"]
taxi_data2020["Zone"] = dfzone["Zone"]
taxi_data2020["service_zone"] = dfzone["service_zone"]

taxi_data2019["Borough"] = dfzone["Borough"]
taxi_data2019["Zone"] = dfzone["Zone"]
taxi_data2019["service_zone"] = dfzone["service_zone"]

In [10]:
biv_colors = [  "#e8e8e8", "#b5c0da", "#6c83b5",
                "#b8d6be", "#90b2b3", "#567994",
                "#73ae80", "#5a9178", "#2a5a5b"]
yellow_percentiles = np.percentile(taxi_data2020["yellow_log_total_amount"], [33, 66])
green_percentiles = np.percentile(taxi_data2020["green_log_total_amount"], [33, 66])
colors = data2color( taxi_data2020["yellow_log_total_amount"],
                        taxi_data2020["green_log_total_amount"],
                        a=yellow_percentiles[0],  b=yellow_percentiles[1], 
                        c=green_percentiles[0],  d=green_percentiles[1],
                        biv_colors=biv_colors)
taxi_data2020["biv_amount_color"] = colors

yellow_percentiles = np.percentile(taxi_data2019["yellow_log_total_amount"], [33, 66])
green_percentiles = np.percentile(taxi_data2019["green_log_total_amount"], [33, 66])
colors = data2color( taxi_data2019["yellow_log_total_amount"],
                        taxi_data2019["green_log_total_amount"],
                        a=yellow_percentiles[0],  b=yellow_percentiles[1], 
                        c=green_percentiles[0],  d=green_percentiles[1],
                        biv_colors=biv_colors)
taxi_data2019["biv_amount_color"] = colors

In [11]:
# Drop unnecessary columns
taxi_data2020.drop(columns=["yellow_log_total_amount", "green_log_total_amount"], inplace=True)
taxi_data2019.drop(columns=["yellow_log_total_amount", "green_log_total_amount"], inplace=True)

In [12]:
# Ratio is calculated reverse for coloring, it's negative is stored in the output
# So that it represents (2020 - 2019) / 2020
# In words: decrease in revenue from 2019
yellowratio = pd.DataFrame({"ratio":(taxi_data2019["yellow_total_amount"] - taxi_data2020["yellow_total_amount"]) \
                            / taxi_data2019["yellow_total_amount"]}).replace([np.nan, np.inf, -np.inf], 0)
greenratio = pd.DataFrame({"ratio":(taxi_data2019["green_total_amount"] - taxi_data2020["green_total_amount"]) \
                            / taxi_data2019["green_total_amount"]}).replace([np.nan, np.inf, -np.inf], 0)

yellow_percentiles = np.percentile(yellowratio["ratio"], [33, 66])
green_percentiles = np.percentile(greenratio["ratio"], [33, 66])

colors = data2color( yellowratio["ratio"],
                        greenratio["ratio"],
                        a=yellow_percentiles[0],  b=yellow_percentiles[1], 
                        c=green_percentiles[0],  d=green_percentiles[1],
                        biv_colors=biv_colors)
taxi_data2020["yellow_change_percent"] = -yellowratio["ratio"] * 100
taxi_data2020["green_change_percent"] = -greenratio["ratio"] * 100
taxi_data2020["biv_ratio_color"] = colors

In [13]:
taxi_data2020

,PULocationID,yellow_total_amount,green_total_amount,Borough,Zone,service_zone,biv_amount_color,yellow_change_percent,green_change_percent,biv_ratio_color
0,1,31304.97,369.26,EWR,Newark Airport,EWR,#6c83b5,-48.779795,6.154952,#e8e8e8
1,2,9.80,6.59,Queens,Jamaica Bay,Boro Zone,#e8e8e8,-91.977734,-0.000000,#6c83b5
2,3,2697.85,4478.20,Bronx,Allerton/Pelham Gardens,Boro Zone,#b8d6be,-64.362236,-74.641440,#2a5a5b
3,4,96227.39,855.91,Manhattan,Alphabet City,Yellow Zone,#6c83b5,-64.569092,2760.661765,#6c83b5
4,5,533.07,0.00,Staten Island,Arden Heights,Boro Zone,#e8e8e8,-68.819022,-100.000000,#2a5a5b
...,...,...,...,...,...,...,...,...,...,...
260,261,353550.28,826.71,Manhattan,World Trade Center,Yellow Zone,#6c83b5,-64.751364,196.993102,#6c83b5
261,262,688824.37,470.15,Manhattan,Yorkville East,Yellow Zone,#6c83b5,-52.340863,18.267804,#b5c0da
262,263,979706.53,3002.55,Manhattan,Yorkville West,Yellow Zone,#567994,-54.815698,-36.222615,#b5c0da
263,264,408870.17,3969.16,Unknown,NV,NaN,#567994,-60.655386,-83.340734,#5a9178


In [14]:
taxi_data2019

,PULocationID,yellow_total_amount,green_total_amount,Borough,Zone,service_zone,biv_amount_color
0,1,61118.40,347.85,EWR,Newark Airport,EWR,#6c83b5
1,2,122.16,0.00,Queens,Jamaica Bay,Boro Zone,#e8e8e8
2,3,7570.20,17659.52,Bronx,Allerton/Pelham Gardens,Boro Zone,#b8d6be
3,4,271591.66,29.92,Manhattan,Alphabet City,Yellow Zone,#6c83b5
4,5,1709.60,3410.34,Staten Island,Arden Heights,Boro Zone,#b8d6be
...,...,...,...,...,...,...,...
260,261,1003018.34,278.36,Manhattan,World Trade Center,Yellow Zone,#6c83b5
261,262,1445314.41,397.53,Manhattan,Yorkville East,Yellow Zone,#6c83b5
262,263,2168245.38,4707.86,Manhattan,Yorkville West,Yellow Zone,#567994
263,264,1039202.40,23825.54,Unknown,NV,NaN,#567994


In [15]:
#taxi_data2020.to_csv("taxi_data_2020-3.csv", encoding='utf-8', index=False)
#taxi_data2019.to_csv("taxi_data_2019-3.csv", encoding='utf-8', index=False)

In [16]:
def ProcessAllTaxiData():
    for monthID in tqdm(range(1, 13)):   
        # Read the dataset into a pandas dataframe
        dfyellow2020 = pd.read_csv("rawdata/yellow_tripdata_2020-" + f"{monthID:02}" + ".csv", \
                    dtype={'PULocationID': int, 'total_amount': float}, \
                    usecols=['PULocationID', 'total_amount'])

        dfgreen2020 = pd.read_csv("rawdata/green_tripdata_2020-" + f"{monthID:02}" + ".csv", \
                    dtype={'PULocationID': int, 'total_amount': float}, \
                    usecols=['PULocationID', 'total_amount'])

        dfyellow2019 = pd.read_csv("rawdata/yellow_tripdata_2019-" + f"{monthID:02}" + ".csv", \
                    dtype={'PULocationID': int, 'total_amount': float}, \
                    usecols=['PULocationID', 'total_amount'])

        dfgreen2019 = pd.read_csv("rawdata/green_tripdata_2019-" + f"{monthID:02}" + ".csv", \
                    dtype={'PULocationID': int, 'total_amount': float}, \
                    usecols=['PULocationID', 'total_amount'])
        
        dfzone = pd.read_csv('taxi_data/taxi_zone_lookup.csv')
        
        taxi_data2020 = dfyellow2020.groupby(['PULocationID']).sum().reindex(range(1, len(dfzone)+1)).fillna(0)
        green_data2020 = dfgreen2020.groupby(['PULocationID']).sum().reindex(range(1, len(dfzone)+1)).fillna(0)
        taxi_data2019 = dfyellow2019.groupby(['PULocationID']).sum().reindex(range(1, len(dfzone)+1)).fillna(0)
        green_data2019 = dfgreen2019.groupby(['PULocationID']).sum().reindex(range(1, len(dfzone)+1)).fillna(0)
        
        def myLog(x):
            return np.log(x) if (x > 0) else 0

        #taxi_data['PULocationID'] = taxi_data.index
        taxi_data2020.reset_index(level=0, inplace=True)
        taxi_data2020["log_total_amount"] = np.array(list(map(myLog, taxi_data2020["total_amount"])))

        green_data2020.reset_index(level=0, inplace=True)
        green_data2020["log_total_amount"] = np.array(list(map(myLog, green_data2020["total_amount"])))

        taxi_data2019.reset_index(level=0, inplace=True)
        taxi_data2019["log_total_amount"] = np.array(list(map(myLog, taxi_data2019["total_amount"])))

        green_data2019.reset_index(level=0, inplace=True)
        green_data2019["log_total_amount"] = np.array(list(map(myLog, green_data2019["total_amount"])))
        
        taxi_data2020["green_total_amount"] = green_data2020["total_amount"]
        taxi_data2020["green_log_total_amount"] = green_data2020["log_total_amount"]
        taxi_data2020.rename(columns={
                            "total_amount": "yellow_total_amount",
                            "log_total_amount": "yellow_log_total_amount"
                        }, inplace=True)

        taxi_data2019["green_total_amount"] = green_data2019["total_amount"]
        taxi_data2019["green_log_total_amount"] = green_data2019["log_total_amount"]
        taxi_data2019.rename(columns={
                            "total_amount": "yellow_total_amount",
                            "log_total_amount": "yellow_log_total_amount"
                        }, inplace=True)

        taxi_data2020["Borough"] = dfzone["Borough"]
        taxi_data2020["Zone"] = dfzone["Zone"]
        taxi_data2020["service_zone"] = dfzone["service_zone"]

        taxi_data2019["Borough"] = dfzone["Borough"]
        taxi_data2019["Zone"] = dfzone["Zone"]
        taxi_data2019["service_zone"] = dfzone["service_zone"]
        
        biv_colors = [  "#e8e8e8", "#b5c0da", "#6c83b5",
                "#b8d6be", "#90b2b3", "#567994",
                "#73ae80", "#5a9178", "#2a5a5b"]
        yellow_percentiles = np.percentile(taxi_data2020["yellow_log_total_amount"], [33, 66])
        green_percentiles = np.percentile(taxi_data2020["green_log_total_amount"], [33, 66])
        colors = data2color( taxi_data2020["yellow_log_total_amount"],
                                taxi_data2020["green_log_total_amount"],
                                a=yellow_percentiles[0],  b=yellow_percentiles[1], 
                                c=green_percentiles[0],  d=green_percentiles[1],
                                biv_colors=biv_colors)
        taxi_data2020["biv_amount_color"] = colors

        yellow_percentiles = np.percentile(taxi_data2019["yellow_log_total_amount"], [33, 66])
        green_percentiles = np.percentile(taxi_data2019["green_log_total_amount"], [33, 66])
        colors = data2color( taxi_data2019["yellow_log_total_amount"],
                                taxi_data2019["green_log_total_amount"],
                                a=yellow_percentiles[0],  b=yellow_percentiles[1], 
                                c=green_percentiles[0],  d=green_percentiles[1],
                                biv_colors=biv_colors)
        taxi_data2019["biv_amount_color"] = colors
        
        # Drop unnecessary columns
        taxi_data2020.drop(columns=["yellow_log_total_amount", "green_log_total_amount"], inplace=True)
        taxi_data2019.drop(columns=["yellow_log_total_amount", "green_log_total_amount"], inplace=True)
        
        # Ratio is calculated reverse for coloring, it's negative is stored in the output
        # So that it represents (2020 - 2019) / 2020
        # In words: decrease in revenue from 2019
        yellowratio = pd.DataFrame( \
                        {"ratio":(taxi_data2019["yellow_total_amount"] - taxi_data2020["yellow_total_amount"]) \
                                    / taxi_data2019["yellow_total_amount"]}).replace([np.nan, np.inf, -np.inf], 0)
        greenratio = pd.DataFrame( \
                        {"ratio":(taxi_data2019["green_total_amount"] - taxi_data2020["green_total_amount"]) \
                                    / taxi_data2019["green_total_amount"]}).replace([np.nan, np.inf, -np.inf], 0)
        
        yellow_percentiles = np.percentile(yellowratio["ratio"], [33, 66])
        green_percentiles = np.percentile(greenratio["ratio"], [33, 66])

        colors = data2color( yellowratio["ratio"],
                                greenratio["ratio"],
                                a=yellow_percentiles[0],  b=yellow_percentiles[1], 
                                c=green_percentiles[0],  d=green_percentiles[1],
                                biv_colors=biv_colors)
        taxi_data2020["yellow_change_percent"] = -yellowratio["ratio"] * 100
        taxi_data2020["green_change_percent"] = -greenratio["ratio"] * 100
        taxi_data2020["biv_ratio_color"] = colors
        
        taxi_data2020.to_csv("taxi_data_2020-" + f"{monthID:02}" + ".csv", encoding='utf-8', index=False)
        taxi_data2019.to_csv("taxi_data_2019-" + f"{monthID:02}" + ".csv", encoding='utf-8', index=False)

In [17]:
ProcessAllTaxiData()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [02:09<00:00, 10.83s/it]
